## Calcular el Hand Completo para toda una zona

In [ ]:
# ------------------------------------------------------------------------------------------------------------------
# Funcion para crear raster a partir de un array

# Importar todas las  librerías
def array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array):

    import gdal, ogr, os, osr
    
    cols = array.shape[1]
    rows = array.shape[0]
    originX = rasterOrigin[0]
    originY = rasterOrigin[1]

    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Int16, ['COMPRESS=DEFLATE','PREDICTOR=2','ZLEVEL=1'])
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.SetNoDataValue(-9999.)  #ensayo
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(4326)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

In [ ]:
# Importar todas las  librerías

from osgeo import gdal, gdalconst
from pysheds.grid import Grid
import numpy as np
import fnmatch
import glob, os
import shutil
import re
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import matplotlib.pyplot as plt
import matplotlib.colors as colors


In [ ]:
%%time

Pais = 'Mexico'

# Coordenadas del dominio espacial que abarca todas las planchas que componen la región de interés.
# Lat_infer_izq, Lon_infer_izq, Lat_sup_der, Lon_sup_der 

dominio = '22,-107,32,-97'
hand_name = 'HAND_N22W107_N32W97'

# Extension del archivo que contiene los datos del DEM. Puede ser Tiff, o bil, o ascii, o cualquier formato de estos
extension = 'bil'

# Tamaño del pixel de salida del HAND. Se entra argumento para remuestrear y hacer la salida menos pesada para el
# procesamiento
size_pixel = 200


# ------------------------------------------------------------------------------------------------------------------
# Preparar los directorios de trabajo

# Ruta absoluta donde están guardados los dems que bajamos para toda la región del USGS. A menos que se quiera
#  procesar una región diferente a las planchas descargadas, este debería quedar por defecto
path_dems = '/opt/geociencia/coreclima/Compartidos/HAND_Region/Procesamiento_DEMs_prueba/1.DEMs_unzip/'

out_ruta = '/opt/geociencia/coreclima/Compartidos/HAND_Region/' + Pais + '/'

if not os.path.isdir(out_ruta):
    try:
        os.mkdir(out_ruta)
    except OSError:
        print ("Creation of the directory %s failed" % out_ruta)
    else:
        print ("Successfully created the directory %s " % out_ruta)    


# ------------------------------------------------------------------------------------------------------------------        
# Preprarar la lista con el nombre de los archivos del dominio a procesar

dominio = list(map(float, dominio.split(',')))
os.chdir(path_dems)
file_list = list()
for file in glob.glob('*.' + extension):
    file_list.append(file)
file_list.sort()

processlist = list()

for file in file_list:
    coord = re.findall(r'(\d+)_',file)
    coord = list(map(int, coord))
    lat = re.findall(r'(.)\d+_',file)[0]
    if lat == 's':
        coord[0]=-coord[0]
        
    if (dominio[0]-1 <= coord[0] <= dominio[2]+1 and dominio[1]-1 <= -coord[1] <= dominio[3]+1):
        processlist.append(file)
        

# ------------------------------------------------------------------------------------------------------------------
# Remuestrear los raster del dominio de interés para bajar resolución espacial

for file in processlist:

    inputfile = path_dems+'/'+file
    input_ = gdal.Open(inputfile, gdalconst.GA_ReadOnly)
    inputProj = input_.GetProjection()
    inputTrans = input_.GetGeoTransform()
    bandinput = input_.GetRasterBand(1)  

    # Tamaño de celda en raster original (metros)
    cell_size=inputTrans[1]*111100
    # factor de relación entre tamaño viejo y tamaño nuevo
    factor = int(cell_size)/size_pixel
    f1 = factor*input_.RasterXSize
    ncell = int(f1)
    # factor corregido
    factor_corr = ncell/input_.RasterXSize
    # Resultado
    res = inputTrans[1]/factor_corr

    inputTrans = list(inputTrans)
    inputTrans[1] = res
    inputTrans[-1] = -res
    referenceTrans = tuple(inputTrans)
    x = ncell
    y = ncell


    bandreference = bandinput
    referenceProj = inputProj

    outputfile = out_ruta+file[:-4]+'_resize.tif'
    driver= gdal.GetDriverByName('GTiff')
    output = driver.Create(outputfile,x,y,1,bandreference.DataType)
    output.SetGeoTransform(referenceTrans)
    output.SetProjection(referenceProj)

    gdal.ReprojectImage(input_,output,inputProj,referenceProj,gdalconst.GRA_Bilinear)
    del output

In [ ]:
# ------------------------------------------------------------------------------------------------------------------
# Hacer el merge de todos los raster remuestreados que quedaron dentro del dominio especificado

search_criteria = "*_resize.tif"
q = os.path.join(out_ruta, search_criteria)
dem_fps = glob.glob(q)

src_files_to_mosaic = []
for fp in dem_fps:
    src = rasterio.open(fp)
    src_files_to_mosaic.append(src)

    
mosaic, out_trans = merge(src_files_to_mosaic)

GT_ET=[out_trans.c, out_trans.a, out_trans.b, out_trans.f, out_trans.d, out_trans.e]

# Guardar el Hand como raster
# GT_ET = inputTrans 
rasterOrigin = (GT_ET[0],GT_ET[3])
pixelWidth = GT_ET[1]
pixelHeight = GT_ET[-1]
newRasterfn = out_ruta+'Resize_completo.tif'
array = mosaic[0,:,:] 
array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array)

for i in dem_fps:
    os.remove(i)


In [ ]:
%%time
# ------------------------------------------------------------------------------------------------------------------
# Estimar el HAND del dominio especificado a partir de Pysheds


# Cargar el DEM
grid = Grid.from_raster(out_ruta+'Resize_completo.tif', data_name='dem')
# Fill depressions in DEM
grid.fill_depressions('dem', out_name='flooded_dem')
# Resolve flats in DEM
grid.resolve_flats('flooded_dem', out_name='inflated_dem')
# Specify directional mapping
dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
# Compute flow directions
grid.flowdir(data='inflated_dem', out_name='dir', dirmap=dirmap)
# Calculate flow accumulation
grid.accumulation(data='dir', dirmap=dirmap, out_name='acc')
# Calcula la red de drenaje
R_drain    = grid.view('acc')
R_drain[R_drain<1000]  = 0
R_drain[R_drain>=1000] = 1
# Calcular Hand
grid.compute_hand('dir', 'dem', R_drain, out_name='hand')
grid.hand[grid.hand<0]  = 0

os.remove(out_ruta+'Resize_completo.tif')

In [ ]:
# ------------------------------------------------------------------------------------------------------------------
# Guardar el HAND recortandolo para que no quede con los bordes comidos

rasterOrigin = (GT_ET[0],GT_ET[3])
pixelWidth = GT_ET[1]
pixelHeight = GT_ET[-1]
newRasterfn = out_ruta+'HAND_completo.tif'
array = grid.hand
array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array)

In [ ]:
projwin = [dominio[1],dominio[2],dominio[3],dominio[0]]

translateoptions = gdal.TranslateOptions(
    gdal.ParseCommandLine('-co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=1'), projWin = projwin)

ds = gdal.Open(out_ruta+'HAND_completo.tif')
ds = gdal.Translate(out_ruta+hand_name+'.tif', ds, options=translateoptions)
ds = None

os.remove(out_ruta+'HAND_completo.tif')

In [ ]:
# -------------------------------------------------------------------------------------------------------------------
# Hacer el merge de las planchas hand procesadas. Solo se ejecuta al final cuando se tengan listos todos los
# dominios que integran el país o la región de estudio

search_criteria = "*.tif"
q = os.path.join(out_ruta, search_criteria)
dem_fps = glob.glob(q)

src_files_to_mosaic = []
for fp in dem_fps:
    src = rasterio.open(fp)
    src_files_to_mosaic.append(src)

    
mosaic, out_trans = merge(src_files_to_mosaic)

GT_ET=[out_trans.c, out_trans.a, out_trans.b, out_trans.f, out_trans.d, out_trans.e]

# Guardar el Hand como raster
# GT_ET = inputTrans 
rasterOrigin = (GT_ET[0],GT_ET[3])
pixelWidth = GT_ET[1]
pixelHeight = GT_ET[-1]
newRasterfn = out_ruta + Pais + str(size_pixel) +'m_Python.tif'
array = mosaic[0,:,:] 
array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array)

##  Integrado todo algoritmo para sudamerica 200m

In [7]:
# ------------------------------------------------------------------------------------------------------------------
# Funcion para crear raster a partir de un array

# Importar todas las  librerías
def array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array):

    import gdal, ogr, os, osr
    
    cols = array.shape[1]
    rows = array.shape[0]
    originX = rasterOrigin[0]
    originY = rasterOrigin[1]

    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Int16, ['COMPRESS=DEFLATE','PREDICTOR=2','ZLEVEL=1'])
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.SetNoDataValue(-9999.)  #ensayo
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(4326)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()
    
# Importar todas las  librerías

from osgeo import gdal, gdalconst
from pysheds.grid import Grid
import numpy as np
import fnmatch
import glob, os
import shutil
import re
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# Extension del archivo que contiene los datos del DEM. Puede ser Tiff, o bil, o ascii, o cualquier formato de estos
extension = 'bil'

# Tamaño del pixel de salida del HAND. Se entra argumento para remuestrear y hacer la salida menos pesada para el
# procesamiento
size_pixel = 200

Pais = 'America'


# ------------------------------------------------------------------------------------------------------------------
# Preparar los directorios de trabajo

# Ruta absoluta donde están guardados los dems que bajamos para toda la región del USGS. A menos que se quiera
#  procesar una región diferente a las planchas descargadas, este debería quedar por defecto
path_dems = '/opt/geociencia/coreclima/Compartidos/HAND_Region/Procesamiento_DEMs_prueba/1.DEMs_unzip/'

out_ruta = '/opt/geociencia/coreclima/Compartidos/HAND_Region/' + Pais + '/'

if not os.path.isdir(out_ruta):
    try:
        os.mkdir(out_ruta)
    except OSError:
        print ("Creation of the directory %s failed" % out_ruta)
    else:
        print ("Successfully created the directory %s " % out_ruta)    



# Coordenadas del dominio espacial que abarca todas las planchas que componen la región de interés.
# Lat_infer_izq, Lon_infer_izq, Lat_sup_der, Lon_sup_der 
# Leer los dominios a procesar
Path = '/opt/geociencia/coreclima/Compartidos/HAND_Region/'
file = 'Regiones_200m.csv'

dominios = np.genfromtxt(Path + file, delimiter = ';', skip_header=1)

def sign(s):
    if s < 0:
        lat_ = 'S'
    else:
        lat_ = 'N'
    return lat_

for i in range(len(dominios)):
    coords = list(dominios[i][1:])
    string_coord = [str(cord) for cord in coords]
    dominio = ",".join(string_coord)
    hand_name = 'HAND_'+sign(coords[0])+str(round(coords[0]))+'W'+str(abs(round(coords[1])))+'_'+sign(coords[2])+str(round(coords[2]))+'W'+str(abs(round(coords[3])))

    # ------------------------------------------------------------------------------------------------------------------        
    # Preprarar la lista con el nombre de los archivos del dominio a procesar

    dominio = list(map(float, dominio.split(',')))
    os.chdir(path_dems)
    file_list = list()
    for file in glob.glob('*.' + extension):
        file_list.append(file)
    file_list.sort()

    processlist = list()

    for file in file_list:
        coord = re.findall(r'(\d+)_',file)
        coord = list(map(int, coord))
        lat = re.findall(r'(.)\d+_',file)[0]
        if lat == 's':
            coord[0]=-coord[0]

        if (dominio[0]-1 <= coord[0] <= dominio[2]+1 and dominio[1]-1 <= -coord[1] <= dominio[3]+1):
            processlist.append(file)


    # ------------------------------------------------------------------------------------------------------------------
    # Remuestrear los raster del dominio de interés para bajar resolución espacial

    for file in processlist:

        inputfile = path_dems+'/'+file
        input_ = gdal.Open(inputfile, gdalconst.GA_ReadOnly)
        inputProj = input_.GetProjection()
        inputTrans = input_.GetGeoTransform()
        bandinput = input_.GetRasterBand(1)  

        # Tamaño de celda en raster original (metros)
        cell_size=inputTrans[1]*111100
        # factor de relación entre tamaño viejo y tamaño nuevo
        factor = int(cell_size)/size_pixel
        f1 = factor*input_.RasterXSize
        ncell = int(f1)
        # factor corregido
        factor_corr = ncell/input_.RasterXSize
        # Resultado
        res = inputTrans[1]/factor_corr

        inputTrans = list(inputTrans)
        inputTrans[1] = res
        inputTrans[-1] = -res
        referenceTrans = tuple(inputTrans)
        x = ncell
        y = ncell


        bandreference = bandinput
        referenceProj = inputProj

        outputfile = out_ruta+file[:-4]+'_resize.tif'
        driver= gdal.GetDriverByName('GTiff')
        output = driver.Create(outputfile,x,y,1,bandreference.DataType)
        output.SetGeoTransform(referenceTrans)
        output.SetProjection(referenceProj)

        gdal.ReprojectImage(input_,output,inputProj,referenceProj,gdalconst.GRA_Bilinear)
        del output



    # ------------------------------------------------------------------------------------------------------------------
    # Hacer el merge de todos los raster remuestreados que quedaron dentro del dominio especificado

    search_criteria = "*_resize.tif"
    q = os.path.join(out_ruta, search_criteria)
    dem_fps = glob.glob(q)

    src_files_to_mosaic = []
    for fp in dem_fps:
        src = rasterio.open(fp)
        src_files_to_mosaic.append(src)


    mosaic, out_trans = merge(src_files_to_mosaic)

    GT_ET=[out_trans.c, out_trans.a, out_trans.b, out_trans.f, out_trans.d, out_trans.e]

    # Guardar el Hand como raster
    # GT_ET = inputTrans 
    rasterOrigin = (GT_ET[0],GT_ET[3])
    pixelWidth = GT_ET[1]
    pixelHeight = GT_ET[-1]
    newRasterfn = out_ruta+'Resize_completo.tif'
    array = mosaic[0,:,:] 
    array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array)

    for i in dem_fps:
        os.remove(i)


    # ------------------------------------------------------------------------------------------------------------------
    # Estimar el HAND del dominio especificado a partir de Pysheds


    # Cargar el DEM
    grid = Grid.from_raster(out_ruta+'Resize_completo.tif', data_name='dem')
    # Fill depressions in DEM
    grid.fill_depressions('dem', out_name='flooded_dem')
    # Resolve flats in DEM
    grid.resolve_flats('flooded_dem', out_name='inflated_dem')
    # Specify directional mapping
    dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
    # Compute flow directions
    grid.flowdir(data='inflated_dem', out_name='dir', dirmap=dirmap)
    # Calculate flow accumulation
    grid.accumulation(data='dir', dirmap=dirmap, out_name='acc')
    # Calcula la red de drenaje
    R_drain    = grid.view('acc')
    R_drain[R_drain<1000]  = 0
    R_drain[R_drain>=1000] = 1
    # Calcular Hand
    grid.compute_hand('dir', 'dem', R_drain, out_name='hand')
    grid.hand[grid.hand<0]  = 0

    os.remove(out_ruta+'Resize_completo.tif')

    # ------------------------------------------------------------------------------------------------------------------
    # Guardar el HAND recortandolo para que no quede con los bordes comidos

    rasterOrigin = (GT_ET[0],GT_ET[3])
    pixelWidth = GT_ET[1]
    pixelHeight = GT_ET[-1]
    newRasterfn = out_ruta+'HAND_completo.tif'
    array = grid.hand
    array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array)

    projwin = [dominio[1],dominio[2],dominio[3],dominio[0]]

    translateoptions = gdal.TranslateOptions(
        gdal.ParseCommandLine('-co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=1'), projWin = projwin)

    ds = gdal.Open(out_ruta+'HAND_completo.tif')
    ds = gdal.Translate(out_ruta+hand_name+'.tif', ds, options=translateoptions)
    ds = None

    os.remove(out_ruta+'HAND_completo.tif')

In [8]:
# -------------------------------------------------------------------------------------------------------------------
# Hacer el merge de las planchas hand procesadas. Solo se ejecuta al final cuando se tengan listos todos los
# dominios que integran el país o la región de estudio

search_criteria = "*.tif"
q = os.path.join(out_ruta, search_criteria)
dem_fps = glob.glob(q)

src_files_to_mosaic = []
for fp in dem_fps:
    src = rasterio.open(fp)
    src_files_to_mosaic.append(src)

    
mosaic, out_trans = merge(src_files_to_mosaic)

GT_ET=[out_trans.c, out_trans.a, out_trans.b, out_trans.f, out_trans.d, out_trans.e]

# Guardar el Hand como raster
# GT_ET = inputTrans 
rasterOrigin = (GT_ET[0],GT_ET[3])
pixelWidth = GT_ET[1]
pixelHeight = GT_ET[-1]
newRasterfn = out_ruta + Pais + str(size_pixel) +'m_Python.tif'
array = mosaic[0,:,:] 
array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array)

In [9]:
file_list

['n00_w050_1arc_v3.bil',
 'n00_w051_1arc_v3.bil',
 'n00_w052_1arc_v3.bil',
 'n00_w053_1arc_v3.bil',
 'n00_w054_1arc_v3.bil',
 'n00_w055_1arc_v3.bil',
 'n00_w056_1arc_v3.bil',
 'n00_w057_1arc_v3.bil',
 'n00_w058_1arc_v3.bil',
 'n00_w059_1arc_v3.bil',
 'n00_w060_1arc_v3.bil',
 'n00_w061_1arc_v3.bil',
 'n00_w062_1arc_v3.bil',
 'n00_w063_1arc_v3.bil',
 'n00_w064_1arc_v3.bil',
 'n00_w065_1arc_v3.bil',
 'n00_w066_1arc_v3.bil',
 'n00_w067_1arc_v3.bil',
 'n00_w068_1arc_v3.bil',
 'n00_w069_1arc_v3.bil',
 'n00_w070_1arc_v3.bil',
 'n00_w071_1arc_v3.bil',
 'n00_w072_1arc_v3.bil',
 'n00_w073_1arc_v3.bil',
 'n00_w074_1arc_v3.bil',
 'n00_w075_1arc_v3.bil',
 'n00_w076_1arc_v3.bil',
 'n00_w077_1arc_v3.bil',
 'n00_w078_1arc_v3.bil',
 'n00_w079_1arc_v3.bil',
 'n00_w080_1arc_v3.bil',
 'n00_w081_1arc_v3.bil',
 'n01_w050_1arc_v3.bil',
 'n01_w051_1arc_v3.bil',
 'n01_w052_1arc_v3.bil',
 'n01_w053_1arc_v3.bil',
 'n01_w054_1arc_v3.bil',
 'n01_w055_1arc_v3.bil',
 'n01_w056_1arc_v3.bil',
 'n01_w057_1arc_v3.bil',


In [19]:
's18_w073_1arc_v3_bil' in file_list

False